# Job Salary Prediction
To build a machine learning model capable of predicting job salary and job satisfaction 
we're going to take following approach :
1. Problem definition
2. Data 
3. Evaluation 
4. Features
5. Modeling
6. Experimentation


In [1]:
#import nessary library
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import sklearn as skt
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
#import data
df=pd.read_csv("/content/salary,satisfaction.csv",engine='python',quoting=0)
df.shape

(2946, 83)

In [3]:
dfna=df.fillna(0)  

# Feature in salary predicatin 

2.Business Title \
	Civil Service  Title	 \
  Division/Work Unit \	
  Job Description	 \
  Minimum Qual Requirements	 \
  Preferred Skills	\
  Additional Information \
  Hours/Shift \
  Posting Date \
CommunicationTools	\
TimeFullyProductive	\
EducationTypes \
	SelfTaughtTypes \
  TimeAfterBootcamp	\
  HackathonReasons	\
LanguageWorkedWith	\
	DatabaseWorkedWith	\
  PlatformWorkedWith \
  FrameworkWorkedWith	 \
	IDE	\
  OperatingSystem \
  	Methodology	 \
    VersionControl	\
    CheckInCode \
    officeaddress \

In [4]:
#pre-processing 
#tagging formal education
check=list(dfna.FormalEducation)
FormalEducation_Normalise=[]
for c,obj in enumerate(check):
  if 'Bachelor' in str(obj):
    FormalEducation_Normalise.append('B')
  elif 'Master' in str(obj):
    FormalEducation_Normalise.append('M')
  elif 'without' in str(obj):
    FormalEducation_Normalise.append('WD')
  elif 'Secondary' in str(obj):
    FormalEducation_Normalise.append('SS')
  elif 'Professional' in str(obj):
    FormalEducation_Normalise.append('PD')
  elif 'Associate degree' in str(obj):
    FormalEducation_Normalise.append('AD')
  elif 'Secondary' in str(obj):
    FormalEducation_Normalise.append('SS')
  elif 'Primary' in str(obj):
    FormalEducation_Normalise.append('P')
  elif 'doctoral' in str(obj):
    FormalEducation_Normalise.append('PHD')
  else:
    FormalEducation_Normalise.append(obj)

In [5]:
#updating value
dfna['FormalEducation']=FormalEducation_Normalise

In [6]:
#tagging UndergradMajor
tmp2=list(dfna.UndergradMajor)
tag_ugm=[]
for c,obj in enumerate(tmp2):
  if 'statistics' in str(obj):
  
    tag_ugm.append('MS')
  elif 'Computer' in str(object):
    tag_ugm.append('CS')
  elif 'Computer' in str(obj):
    tag_ugm.append('CS')
  elif 'Information' in str(obj):
    tag_ugm.append('IT')
  elif 'Another' in str(obj):
    tag_ugm.append('E')
  elif 'Fine' in str(obj):
    tag_ugm.append('F')
  elif 'natural' in str(obj):
    tag_ugm.append('S')
  elif 'business' in str(obj):
    tag_ugm.append('C')
  elif 'humanitites' in str(obj):
    tag_ugm.append('H')
  else:
    tag_ugm.append(obj)




In [7]:
#updating value
dfna['UndergradMajor']=tag_ugm

In [8]:
#taggin distros
tmp11=list(dfna.OperatingSystem)
tag_os=[]
for c,obj in enumerate(tmp11):
  if 'L' in str(obj):
    tag_os.append('L')
  elif 'W' in str(obj):
    tag_os.append('W')
  elif 'Mac' in str(obj):
    tag_os.append('M')
  elif "arch" in str(obj):
    tag_os.append('ar')
  else:
    tag_os.append(obj)

In [9]:
#updating value
dfna['OperatingSystem']=tag_os

In [10]:
#tagging states
dfs=pd.read_csv('/content/states.csv',engine='python',quoting=0)
newl=list(dfna.officeaddress)
orgl=list(dfna.officeaddress)
  
state=list(dfs.State)
abbr=list(dfs.Abbreviation)
for i in range(len(orgl)):
  for j in range(len(state)):
      if str(state[j]) in str(orgl[i]):
        newl[i]=str(abbr[j])
      
     
dfna['officeaddress']=newl

In [11]:
#cleaning
tmp5=list(dfna.term)
term_fil=[]
for i,obj in enumerate(tmp5):
  term_fil.append(re.sub("[a-z]",'',str(obj)))
dfna['term']=term_fil

tmp4=list(dfna.CompanySize)
CompanySize_fil=[]
for i,obj in enumerate(tmp4):
  tmp=str(obj).replace('to','-')
  CompanySize_fil.append(re.sub("[a-z]",'',tmp))
dfna['CompanySize']=CompanySize_fil

tmp3=list(dfna.YearsCoding)
year_coding_fil=[]
for i,obj in enumerate(tmp3):
  year_coding_fil.append(re.sub("[a-z]",'',str(obj)))
dfna['YearsCoding']=year_coding_fil 

tmp6=list(dfna.Age)
age_fil=[]
for i,obj in enumerate(tmp6):
  age_fil.append(re.sub("[a-z]",'',str(obj)))
dfna['Age']=age_fil

tmp7=list(dfna.emp_length)
emp_length_fil=[]
for i,obj in enumerate(tmp7):
  emp_length_fil.append(re.sub("[a-z]",'',str(obj)))
dfna['emp_length']=emp_length_fil

tmp8=list(dfna.zip_code)
zip_code_fil=[]
for i,obj in enumerate(tmp8):
  zip_code_fil.append(re.sub("[a-z]",'',str(obj)))
dfna['zip_code']=zip_code_fil

tmp9=list(dfna['Business Title'])
Bussiness_title_fil=[]
for i,obj in enumerate(tmp9):
  Bussiness_title_fil.append(re.sub("\(\,.*$",'',str(obj)))
dfna['Business Title']=Bussiness_title_fil

tmp10=list(dfna['Civil Service Title'])
civil_title_fil=[]
for i,obj in enumerate(tmp10):
  civil_title_fil.append(re.sub("\(.*$",'',str(obj)))
dfna['Civil Service Title']=civil_title_fil

tmpprof=list(dfna.YearsCodingProf)
year_coding_prof=[]
for i,obj in enumerate(tmpprof):
  year_coding_prof.append(re.sub("[a-z]",'',str(obj)))
dfna['YearsCodingProf']=year_coding_prof

In [12]:
#onehot_encode
def onehot_encode(dfna,column):
  dfna=dfna.copy()
  dummies=pd.get_dummies(dfna[column],prefix=column)
  dfna=pd.concat([dfna,dummies],axis=1)
  dfna=dfna.drop(column,axis=1)
  return dfna


In [13]:
#dropping dummy column
dfna=dfna.drop(['AIDangerous','AIFuture','AIInteresting','AIResponsible','Additional Information','AgreeDisagree1','AgreeDisagree3','CheckInCode','AgreeDisagree2','DatabaseDesireNextYear'
,'CrimeTime','Email','JobSearchStatus','LOAN_AMT','id','Target_Salary','Target_Satisfied','Job Description','To Apply','Residency Requirement',	'Posting Date','UpdateCV','HackathonReasons','FrameworkDesireNextYear',
           'loan_status',	'pymnt_plan','orignal_website_directory','purpose','title','zip_code','addr_state','dti','earliest_cr_line','last_pymnt_d',
           'last_pymnt_amnt',	'next_pymnt_d',	'last_credit_pull_d','Time','your_favoritearticle_today',	'url',	'homeaddress',	'latitude',	'longitude',
           'website',	'dateAdded',	'previousaddress',	'phones','LanguageDesireNextYear','Minimum Qual Requirements','PHONE','PlatformDesireNextYear','Posting Updated','Process Date'],axis=1)

In [14]:
#extracting integer value
def extract_int_val(col):
  num=[]
  non=[]
  for w in col:
   w=w.replace(' ','')
   w=w.replace('U','')
   x=(int(w.split('-')[0]))
   z=x
   if '-' in str(w):
      y=(int(w.split('-')[1]))
      z=(x+y)/2
      num.append(z)
   else:
      non.append(w)
      num.append(z)
  return num
    

In [15]:
dfna['Age'].unique()

array(['25 - 34  ', '35 - 44  ', '0', '18 - 24  ', '45 - 54  ',
       '55 - 64  ', 'U 18  '], dtype=object)

In [16]:
age=extract_int_val(list(dfna.Age))
dfna['Age']=age

In [17]:
Yearcodingprof=extract_int_val(list(dfna.YearsCodingProf)) 
dfna['YearsCodingProf']=Yearcodingprof

In [18]:
yearcoding=extract_int_val(list(dfna.YearsCoding))
dfna['YearsCoding']=yearcoding

In [19]:
dfna.dtypes

Business Title          object
Civil Service Title     object
Division/Work Unit      object
Preferred Skills        object
Hours/Shift             object
FormalEducation         object
UndergradMajor          object
CompanySize             object
DevType                 object
YearsCoding            float64
YearsCodingProf        float64
HopeFiveYears           object
LastNewJob              object
CommunicationTools      object
TimeFullyProductive     object
EducationTypes          object
SelfTaughtTypes         object
TimeAfterBootcamp       object
LanguageWorkedWith      object
DatabaseWorkedWith      object
PlatformWorkedWith      object
FrameworkWorkedWith     object
IDE                     object
OperatingSystem         object
Methodology             object
VersionControl          object
Age                    float64
term                    object
emp_length              object
officeaddress           object
dtype: object

In [20]:
def categorical(col):
  for obj in col:
    dfna[col]=pd.Categorical(dfna[col])
categorical(['FormalEducation','OperatingSystem','emp_length'])

In [21]:
#for all categorical fearture
#one hot encode
for column in ['CompanySize','HopeFiveYears','LastNewJob','TimeAfterBootcamp','TimeFullyProductive','UndergradMajor','FormalEducation','officeaddress','DevType','EducationTypes','PlatformWorkedWith','OperatingSystem','emp_length']:
  dfna=onehot_encode(dfna,column)

In [22]:
{column: len(dfna[column].unique()) for column in dfna.columns}

{'Business Title': 1244,
 'Civil Service Title': 296,
 'Division/Work Unit': 678,
 'Preferred Skills': 1283,
 'Hours/Shift': 182,
 'YearsCoding': 12,
 'YearsCodingProf': 12,
 'CommunicationTools': 349,
 'SelfTaughtTypes': 227,
 'LanguageWorkedWith': 1579,
 'DatabaseWorkedWith': 692,
 'FrameworkWorkedWith': 217,
 'IDE': 840,
 'Methodology': 156,
 'VersionControl': 50,
 'Age': 7,
 'term': 2,
 'CompanySize_1,000 - 4,999 ': 2,
 'CompanySize_10 - 19 ': 2,
 'CompanySize_10,000   ': 2,
 'CompanySize_100 - 499 ': 2,
 'CompanySize_20 - 99 ': 2,
 'CompanySize_5,000 - 9,999 ': 2,
 'CompanySize_500 - 999 ': 2,
 'CompanySize_F  10 ': 2,
 'HopeFiveYears_0': 2,
 'HopeFiveYears_Doing the same work': 2,
 'HopeFiveYears_Retirement': 2,
 'HopeFiveYears_Working as a founder or co-founder of my own company': 2,
 'HopeFiveYears_Working as a product manager or project manager': 2,
 'HopeFiveYears_Working as an engineering manager or other functional manager': 2,
 'HopeFiveYears_Working in a career completely

In [23]:
for column in ['Business Title','Civil Service Title','CommunicationTools','LanguageWorkedWith','Hours/Shift','IDE','FrameworkWorkedWith','VersionControl','DatabaseWorkedWith','Methodology','Division/Work Unit','Preferred Skills',
 'SelfTaughtTypes']:
  dfna=onehot_encode(dfna,column)

In [24]:
target_Salary=df['Target_Salary']

In [25]:
X=dfna
Y=target_Salary

# Modeling 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=50)
model = RandomForestRegressor(n_estimators=10000 ,n_jobs=-1,random_state=True)
model.fit(X_train,y_train)

In [ ]:
#checking model on tst dataset
#accuracy can be increase by increasig estimator 
from sklearn.metrics import mean_squared_error
y_preds = model.predict(X_test)
rmse = mean_squared_error(y_test,y_preds,squared=False)
rmse

In [ ]:
#checking your model on training dataset
from sklearn.metrics import mean_squared_error
y_preds = model.predict(X_train)
rmse = mean_squared_error(y_train,y_preds,squared=False)
rmse

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
dfn=pd.DataFrame({'dti':df['dti'],'LOAN_AMT':df['LOAN_AMT'],'loan_status':df['loan_status'],
                'purpose':df['purpose'],'title':df['title'],'last_pymnt_amnt':df['last_pymnt_amnt']})

In [ ]:
dfn['Target_Salary']=df['Target_Salary']

In [ ]:
{column: len(dfn[column].unique()) for column in dfn.columns}

In [ ]:
tmp=dfn.LOAN_AMT
new=[]
q=[]
for i,obj in enumerate(tmp):
  l=obj.replace('$','')
  l=l.replace(',','')
  l=l.replace(' ','')
  q.append(l)

In [ ]:
dfn['LOAN_AMT']=q

In [ ]:
dfn['LOAN_AMT']=pd.to_numeric(dfn['LOAN_AMT'],downcast='float')

In [ ]:
#onehot_encode
def onehot_encode(dfn,column):
  dfn=dfn.copy()
  dummies=pd.get_dummies(dfn[column],prefix=column)
  dfn=pd.concat([dfn,dummies],axis=1)
  dfn=dfn.drop(column,axis=1)
  return dfn

In [ ]:
for column in ['loan_status','purpose','title']:
  dfn=onehot_encode(dfn,column)

In [ ]:
dff=pd.concat([dfna,dfn],axis=1)

In [ ]:
X=dff
Z=df['Target_Satisfied']


In [ ]:
X_train,X_test,z_train,z_test=train_test_split(X,Z,test_size=0.2)
class_model=RandomForestClassifier(n_estimators=1000,n_jobs=-1,random_state=100,min_samples_split=3,min_samples_leaf=2)
class_model.fit(X_train,z_train)

In [ ]:
z_pred=class_model.predict(X_test)
cf=confusion_matrix(z_test,z_pred)
cf

In [ ]:
dft=pd.DataFrame({'z_true':z_test,'z_pred':z_pred})
dft.head(50)

In [ ]:
z_val=class_model.predict(X_train)
cf=confusion_matrix(z_val,z_train)
cf

In [ ]:
TP=
TN=
FP=
FN=

In [ ]:
# Sensitivity or Recall
metric = "TPR"
results[metric] = TP / (TP + FN)
print(f"{metric} is {results[metric]: .3f}")

In [ ]:
# Precision
metric = "PPV"
results[metric] = TP / (TP + FP)
print(f"{metric} is {results[metric]: .3f}")

In [ ]:
metric = "F1"
results[metric] = 2 / (1 / results["PPV"] + 1 / results["TPR"])
print(f"{metric} is {results[metric]: .3f}")